In [ ]:
# Kết nối với google drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import asyncio
import aiohttp
import json
import os
import pandas as pd
from datetime import datetime, timedelta
import requests
import time
from tqdm import tqdm
import concurrent.futures

# 1. Đọc dữ liệu

Tại vì api không lấy được hết dữ liệu các tỉnh thành nên ta sẽ lấy dữ liệu của một tỉnh, thành phố làm đại diện cho tỉnh thành đó

Ví Dụ:
- Tỉnh Lâm Đồng chỉ lấy được dữ liệu API từ Đà Lạt mà không lấy được các nơi khác như Bảo Lâm, Bảo Lộc nên ta chỉ lấy dữ liệu Đà Lạt làm đại diện.
- Đối với một vài tỉnh thành như An Giang, Khánh Hòa,.. có thể lấy được dữ liệu của nhiều tỉnh. Nhưng để dữ liệu mang tính đồng nhất với các tỉnh khác. Ta chỉ lấy dữ liệu của một tỉnh trong nó làm đại diện. Trong đó, thành phố hành chính của tỉnh sẽ được chọn.

Dưới đây là các tên *city* được dùng để đại diện cho tỉnh thành của Việt Nam, dựa trên thành phố hành chính của tỉnh.


| STT | Tên Tỉnh Thành         | q (Tên địa phương đại diện) |
|-----|------------------------|-----------------------------|
| 1   | An Giang              | Long Xuyen                  |
| 2   | Bà Rịa-Vũng Tàu       | Vung Tau                    |
| 3   | Bắc Giang             | Bac Giang                   |
| 4   | Bắc Kạn               | Bac Kan                     |
| 5   | Bạc Liêu              | Bac Lieu                    |
| 6   | Bắc Ninh              | Bac Ninh                    |
| 7   | Bến Tre               | Ben Tre                     |
| 8   | Bình Định             | Quy Nhon                    |
| 9   | Bình Dương            | Thu Dau Mot                 |
| 10  | Bình Phước            | Dong Xoai                   |
| 11  | Bình Thuận            | Phan Thiet                  |
| 12  | Cà Mau                | Ca Mau                      |
| 13  | Cần Thơ               | Can Tho                     |
| 14  | Cao Bằng              | Cao Bang                    |
| 15  | Đà Nẵng               | Da Nang                     |
| 16  | Đắk Lắk               | Buon Ma Thuot               |
| 17  | Đắk Nông              | Gia Nghia                   |
| 18  | Điện Biên             | Dien Bien Phu               |
| 19  | Đồng Nai              | Bien Hoa                    |
| 20  | Đồng Tháp             | Cao Lanh                    |
| 21  | Gia Lai               | Pleiku                      |
| 22  | Hà Giang              | Ha Giang                    |
| 23  | Hà Nam                | Phu Ly                      |
| 24  | Hà Nội                | Hanoi                       |
| 25  | Hà Tĩnh               | Ha Tinh                     |
| 26  | Hải Dương             | Hai Duong                   |
| 27  | Hải Phòng             | Hai Phong                   |
| 28  | Hậu Giang             | Vi Thanh                    |
| 29  | TP. Hồ Chí Minh       | Ho Chi Minh City            |
| 30  | Hòa Bình              | Hoa Binh                    |
| 31  | Hưng Yên              | Hung Yen                    |
| 32  | Khánh Hòa             | Nha Trang                   |
| 33  | Kiên Giang            | Rach Gia                    |
| 34  | Kon Tum               | Kon Tum                     |
| 35  | Lai Châu              | Lai Chau                    |
| 36  | Lâm Đồng              | Da Lat                      |
| 37  | Lạng Sơn              | Lang Son                    |
| 38  | Lào Cai               | Lao Cai                     |
| 39  | Long An               | Tan An                      |
| 40  | Nam Định              | Nam Dinh                    |
| 41  | Nghệ An               | Vinh                        |
| 42  | Ninh Bình             | Ninh Binh                   |
| 43  | Ninh Thuận            | Phan Rang-Thap Cham         |
| 44  | Phú Thọ               | Viet Tri                    |
| 45  | Phú Yên               | Tuy Hoa                     |
| 46  | Quảng Bình            | Dong Hoi                    |
| 47  | Quảng Nam             | Hoi An                      |
| 48  | Quảng Ngãi            | Quang Ngai                  |
| 49  | Quảng Ninh            | Ha Long                     |
| 50  | Quảng Trị             | Dong Ha                     |
| 51  | Sóc Trăng             | Soc Trang                   |
| 52  | Sơn La                | Son La                      |
| 53  | Tây Ninh              | Tay Ninh                    |
| 54  | Thái Bình             | Thai Binh                   |
| 55  | Thái Nguyên           | Thai Nguyen                 |
| 56  | Thanh Hóa             | Thanh Hoa                   |
| 57  | Thừa Thiên - Huế      | Hue                         |
| 58  | Tiền Giang            | My Tho                      |
| 59  | Trà Vinh              | Tra Vinh                    |
| 60  | Tuyên Quang           | Tuyen Quang                 |
| 61  | Vĩnh Long             | Vinh Long                   |
| 62  | Vĩnh Phúc             | Vinh Yen                    |
| 63  | Yên Bái               | Yen Bai                     |

In [ ]:
from google.colab import userdata
userdata.get('weatherapi')
#userdata.get('historyhour')
print("Get secret keys successfully")

Get secret keys successfully


In [ ]:
secret_key = userdata.get('weatherapi')
#secret_key = userdata.get('historyhour')

In [ ]:
from datetime import datetime, timedelta
end_date = datetime.today()
start_date = end_date - timedelta(days=366*6)
end_date - timedelta(days=1)

datetime.datetime(2025, 6, 2, 2, 23, 32, 346105)

In [ ]:
# 1. Parameters
api_key = secret_key  # Thay bằng key của bạn
cities = [
    "Long Xuyen", "Vung Tau", "Bac Giang", "Bac Kan", "Bac Lieu", "Bac Ninh", "Ben Tre",
    "Quy Nhon", "Thu Dau Mot", "Dong Xoai", "Phan Thiet", "Ca Mau", "Can Tho", "Cao Bang",
    "Da Nang", "Buon Ma Thuot", "Gia Nghia", "Dien Bien Phu", "Bien Hoa", "Cao Lanh",
    "Pleiku", "Ha Giang", "Phu Ly", "Hanoi", "Ha Tinh", "Hai Duong", "Hai Phong",
    "Vi Thanh", "Ho Chi Minh City", "Hoa Binh", "Hong Yen", "Nha Trang", "Rach Gia",
    "Kon Tum", "Lai Chau", "Da Lat", "Lang Son", "Lao Cai", "Tan An", "Nam Dinh",
    "Vinh", "Ninh Binh", "Phan Rang-Thap Cham", "Viet Tri", "Tuy Hoa", "Dong Hoi",
    "Hoi An", "Quang Ngai", "Ha Long", "Thon Nhu Le", "Soc Trang", "Son La", "Tay Ninh",
    "Thai Binh", "Thai Nguyen", "Thanh Hoa", "Thua Thien Hue", "My Tho", "Tra Vinh", "Tuyen Quang",
    "Vinh Long", "Vinh Yen", "Yen Bai"
]

# Time parameter (from 01/01/2021)
start_date = datetime(2021, 1, 1)
end_date = datetime.today()

# Tạo thư mục output
output_dir = "/content/drive/MyDrive/weather_data"  # Đường dẫn Google Drive
os.makedirs(output_dir, exist_ok=True)

In [ ]:
# 2. Handle fetching data by months
import urllib
async def fetch_month_data(session, city, year, month):
    # Tính ngày đầu và cuối tháng
    if month == 12:
        next_month = 1
        next_year = year + 1
    else:
        next_month = month + 1
        next_year = year

    # Ngày đầu tháng
    start_date = datetime(year, month, 1)
    # Ngày cuối tháng (ngày trước ngày đầu tiên của tháng tiếp theo)
    end_date = datetime(next_year, next_month, 1) - timedelta(days=1)
    # Tên thành phố
    Encoded_city = urllib.parse.quote(city)

    # Giới hạn với ngày hiện tại
    today = datetime.today()
    if end_date > today:
        end_date = today

    # Định dạng ngày thành yyyy-MM-dd
    start_str = start_date.strftime('%Y-%m-%d')
    end_str = end_date.strftime('%Y-%m-%d')

    # Tạo URL với tham số end_dt
    url = f"http://api.weatherapi.com/v1/history.json?key={api_key}&q={Encoded_city}&dt={start_str}&end_dt={end_str}"

    try:
        async with session.get(url) as response:
            if response.status == 200:
                data = await response.json()
                print(f"✅ {city} - {start_str} to {end_str}")
                return data, start_date, end_date
            else:
                print(f"❌ {city} - {start_str} to {end_str} | Lỗi: {response.status}")
                return None, start_date, end_date
    except Exception as e:
        print(f"🚫 Lỗi: {city} - {start_str} to {end_str} - {e}")
        return None, start_date, end_date

In [ ]:
# 3. Handle batch processing by cites + months
async def process_city_data(session, city, semaphore, start_year=2021, start_month=1):
    # Tạo thư mục cho thành phố
    city_dir = f"{output_dir}/{city.replace(' ', '_')}"
    os.makedirs(city_dir, exist_ok=True)

    # Tính toán tháng hiện tại
    today = datetime.today()
    end_year = today.year
    end_month = today.month

    # Lặp qua từng tháng từ start_date đến hiện tại
    for year in range(start_year, end_year + 1):
        # Xác định tháng bắt đầu và kết thúc cho năm hiện tại
        month_start = 1
        month_end = 12

        if year == start_year:
            month_start = start_month
        if year == end_year:
            month_end = end_month

        for month in range(month_start, month_end + 1):
            # Tạo thư mục năm/tháng
            year_month = f"{year}_{month:02d}"
            month_dir = f"{city_dir}/{year_month}"
            os.makedirs(month_dir, exist_ok=True)

            # Tên file
            file_name = f"{month_dir}/monthly_data.json"

            # Kiểm tra nếu file đã tồn tại
            if os.path.exists(file_name) and os.path.getsize(file_name) > 0:
                print(f"Đã tồn tại: {city} - {year_month}")
            else:
                # Lấy dữ liệu với giới hạn tốc độ
                async with semaphore:
                    data, start_date, end_date = await fetch_month_data(session, city, year, month)

                    if data:
                        # Lưu dữ liệu vào file
                        with open(file_name, 'w') as f:
                            json.dump(data, f)

                        # Lưu thông tin metadata
                        meta_file = f"{month_dir}/metadata.json"
                        with open(meta_file, 'w') as f:
                            json.dump({
                                "city": city,
                                "year": year,
                                "month": month,
                                "start_date": start_date.strftime('%Y-%m-%d'),
                                "end_date": end_date.strftime('%Y-%m-%d'),
                                "days_count": (end_date - start_date).days + 1
                            }, f)

                        # Đợi một chút để tránh quá tải API
                        await asyncio.sleep(0.2)

In [ ]:
async def main():
    # Giới hạn số lượng request đồng thời
    semaphore = asyncio.Semaphore(10)  # Điều chỉnh theo giới hạn của API

    async with aiohttp.ClientSession() as session:
        # Xử lý từng nhóm thành phố
        batch_size = 10  # Xử lý 10 thành phố mỗi lần
        for i in range(0, len(cities), batch_size):
            city_batch = cities[i:i+batch_size]

            # Tạo tasks cho tất cả các thành phố trong nhóm
            tasks = []
            for city in city_batch:
                tasks.append(process_city_data(session, city, semaphore))

            # Chạy tất cả tasks
            await asyncio.gather(*tasks)

            print(f"Đã hoàn thành {i+len(city_batch)}/{len(cities)} thành phố")

            # Đợi một chút giữa các batch
            await asyncio.sleep(1)

In [ ]:
# Mapping to region col:
province_mapping = {
    "Long Xuyen": "An Giang",
    "Vung Tau": "Ba Ria-Vung Tau",
    "Bac Giang": "Bac Giang",
    "Bac Kan": "Bac Kan",
    "Bac Lieu": "Bac Lieu",
    "Bac Ninh": "Bac Ninh",
    "Ben Tre": "Ben Tre",
    "Quy Nhon": "Binh Dinh",
    "Thu Dau Mot": "Binh Duong",
    "Dong Xoai": "Binh Phuoc",
    "Phan Thiet": "Binh Thuan",
    "Ca Mau": "Ca Mau",
    "Can Tho": "Can Tho",
    "Cao Bang": "Cao Bang",
    "Da Nang": "Da Nang",
    "Buon Ma Thuot": "Dak Lak",
    "Gia Nghia": "Dak Nong",
    "Dien Bien Phu": "Dien Bien",
    "Bien Hoa": "Dong Nai",
    "Cao Lanh": "Dong Thap",
    "Pleiku": "Gia Lai",
    "Ha Giang": "Ha Giang",
    "Phu Ly": "Ha Nam",
    "Hanoi": "Ha Noi",
    "Ha Tinh": "Ha Tinh",
    "Hai Duong": "Hai Duong",
    "Hai Phong": "Hai Phong",
    "Vi Thanh": "Hau Giang",
    "Ho Chi Minh City": "TP. Ho Chi Minh",
    "Hoa Binh": "Hoa Binh",
    "Hong-Yen": "Hung Yen",
    "Nha Trang": "Khanh Hoa",
    "Rach Gia": "Kien Giang",
    "Kon Tum": "Kon Tum",
    "Lai Chau": "Lai Chau",
    "Da Lat": "Lam Dong",
    "Lang Son": "Lang Son",
    "Lao Cai": "Lao Cai",
    "Tan An": "Long An",
    "Nam Dinh": "Nam Dinh",
    "Vinh": "Nghe An",
    "Ninh Binh": "Ninh Binh",
    "Phan Rang- Thap Cham": "Ninh Thuan",
    "Viet Tri": "Phu Tho",
    "Tuy Hoa": "Phu Yen",
    "Dong Hoi": "Quang Binh",
    "Hoi An": "Quang Nam",
    "Quang Ngai": "Quang Ngai",
    "Ha Long": "Quang Ninh",
    "Thon Nhu Le": "Quang Tri",
    "Soc Trang": "Soc Trang",
    "Son La": "Son La",
    "Tay Ninh": "Tay Ninh",
    "Thai Binh": "Thai Binh",
    "Thai Nguyen": "Thai Nguyen",
    "Thanh Hoa": "Thanh Hoa",
    "Hue": "Thua Thien - Hue",
    "My Tho": "Tien Giang",
    "Tra Vinh": "Tra Vinh",
    "Tuyen Quang": "Tuyen Quang",
    "Vinh Long": "Vinh Long",
    "Vinh Yen": "Vinh Phuc",
    "Yen Bai": "Yen Bai"
}

def combine_to_csv():
    print("Bắt đầu kết hợp dữ liệu thành CSV...")
    all_data = []

    # Duyệt qua tất cả các thư mục thành phố
    for city_folder in os.listdir(output_dir):
        city_path = os.path.join(output_dir, city_folder)
        if not os.path.isdir(city_path):
            continue

        # Duyệt qua tất cả các thư mục năm_tháng
        for year_month in os.listdir(city_path):
            month_path = os.path.join(city_path, year_month)
            if not os.path.isdir(month_path):
                continue

            # Đọc file dữ liệu hàng tháng
            data_file = os.path.join(month_path, "monthly_data.json")
            if os.path.exists(data_file):
                try:
                    with open(data_file, 'r') as f:
                        data = json.load(f)

                    # Trích xuất thông tin
                    location = data.get('location', {})
                    forecast = data.get('forecast', {})
                    forecastday = forecast.get('forecastday', [])
                    city_name = location.get('name')
                    province_name = province_mapping.get(city_name, "")

                    for day in forecastday:
                        day_info = day.get('day', {})
                        astro_info = day.get('astro', {})

                        row = {
                            # Thông tin vị trí
                            'city': city_name,
                            'region': province_name,
                            'country': "Vietnam",
                            'lat': location.get('lat'),
                            'lon': location.get('lon'),

                            # Thông tin ngày
                            'date': day.get('date'),

                            # Nhiệt độ
                            'maxtemp_c': day_info.get('maxtemp_c'),
                            'mintemp_c': day_info.get('mintemp_c'),
                            'avgtemp_c': day_info.get('avgtemp_c'),

                            # Thông tin gió
                            'maxwind_kph': day_info.get('maxwind_kph'),

                            # Lượng mưa và độ ẩm
                            'totalprecip_mm': day_info.get('totalprecip_mm'),
                            'avgvis_km': day_info.get('avgvis_km'),
                            'avghumidity': day_info.get('avghumidity'),

                            # Tia UV
                            'uv': day_info.get('uv'),

                            # Thông tin mặt trời và mặt trăng
                            'sunrise': astro_info.get('sunrise'),
                            'sunset': astro_info.get('sunset'),

                        }
                        all_data.append(row)
                except Exception as e:
                    print(f"Lỗi đọc file {data_file}: {e}")

    if all_data:
        df = pd.DataFrame(all_data)
        csv_file = f"{output_dir}/weather_data_combined.csv"
        df.to_csv(csv_file, index=False)
        print(f"Đã lưu {len(all_data)} bản ghi vào file {csv_file}")
    else:
        print("Không có dữ liệu để lưu")

In [ ]:
import nest_asyncio
nest_asyncio.apply()
start_time = time.time()

# Bước 1: Lấy dữ liệu thời tiết
print("Bắt đầu lấy dữ liệu thời tiết...")
asyncio.run(main())

# Bước 2: Kết hợp dữ liệu thành CSV
print("\nHoàn thành lấy dữ liệu, bắt đầu kết hợp thành CSV...")
combine_to_csv()

# Hiển thị thời gian thực hiện
elapsed_time = time.time() - start_time
print(f"Tổng thời gian thực hiện: {elapsed_time:.2f} giây ({elapsed_time/3600:.2f} giờ)")

Bắt đầu lấy dữ liệu thời tiết...
Đã tồn tại: Long Xuyen - 2021_01
Đã tồn tại: Long Xuyen - 2021_02
Đã tồn tại: Long Xuyen - 2021_03
Đã tồn tại: Long Xuyen - 2021_04
Đã tồn tại: Long Xuyen - 2021_05
Đã tồn tại: Long Xuyen - 2021_06
Đã tồn tại: Long Xuyen - 2021_07
Đã tồn tại: Long Xuyen - 2021_08
Đã tồn tại: Long Xuyen - 2021_09
Đã tồn tại: Long Xuyen - 2021_10
Đã tồn tại: Long Xuyen - 2021_11
Đã tồn tại: Long Xuyen - 2021_12
Đã tồn tại: Long Xuyen - 2022_01
Đã tồn tại: Long Xuyen - 2022_02
Đã tồn tại: Long Xuyen - 2022_03
Đã tồn tại: Long Xuyen - 2022_04
Đã tồn tại: Long Xuyen - 2022_05
Đã tồn tại: Long Xuyen - 2022_06
Đã tồn tại: Long Xuyen - 2022_07
Đã tồn tại: Long Xuyen - 2022_08
Đã tồn tại: Long Xuyen - 2022_09
Đã tồn tại: Long Xuyen - 2022_10
Đã tồn tại: Long Xuyen - 2022_11
Đã tồn tại: Long Xuyen - 2022_12
Đã tồn tại: Long Xuyen - 2023_01
Đã tồn tại: Long Xuyen - 2023_02
Đã tồn tại: Long Xuyen - 2023_03
Đã tồn tại: Long Xuyen - 2023_04
Đã tồn tại: Long Xuyen - 2023_05
Đã tồn tại

In [ ]:
import asyncio
import aiohttp
import json
import os
import pandas as pd
from datetime import datetime, timedelta
import time

output_dir = "/content/drive/MyDrive/weather_data"
data_json()

Bắt đầu gộp tất cả file monthly_data.json...
Đã xử lý 100 file (0.53s)
Đã xử lý 200 file (0.98s)
Đã xử lý 300 file (1.41s)
Đã xử lý 400 file (1.93s)
Đã xử lý 500 file (2.39s)
Đã xử lý 600 file (2.82s)
Đã xử lý 700 file (3.29s)
Đã xử lý 800 file (3.73s)
Đã xử lý 900 file (4.23s)
Đã xử lý 1000 file (4.74s)
Đã xử lý 1100 file (5.23s)
Đã xử lý 1200 file (5.78s)
Đã xử lý 1300 file (6.34s)
Đã xử lý 1400 file (6.86s)
Đã xử lý 1500 file (7.60s)
Đã xử lý 1600 file (8.14s)
Đã xử lý 1700 file (8.68s)
Đã xử lý 1800 file (9.20s)
Đã xử lý 1900 file (9.72s)
Đã xử lý 2000 file (10.25s)
Đã xử lý 2100 file (10.91s)
Đã xử lý 2200 file (11.58s)
Đã xử lý 2300 file (12.19s)
Đã xử lý 2400 file (12.80s)
Đã xử lý 2500 file (13.43s)
Đã xử lý 2600 file (14.27s)
Đã xử lý 2700 file (14.74s)
Đã xử lý 2800 file (15.18s)
Đã xử lý 2900 file (15.64s)
Đã xử lý 3000 file (16.10s)
Đã xử lý 3100 file (16.55s)
Đã xử lý 3200 file (16.99s)
Đã xử lý 3300 file (17.45s)
Đã xử lý 3400 file (17.97s)
Đang ghi dữ liệu vào file...
Đã

In [ ]:
def sort_csv_by_date():
    print("Bắt đầu sắp xếp file CSV theo cột date...")
    start_time = time.time()

    # Đường dẫn file CSV
    csv_file = f"{output_dir}/weather_data_combined.csv"

    # Kiểm tra xem file có tồn tại không
    if not os.path.exists(csv_file):
        print(f"Không tìm thấy file {csv_file}")
        return

    try:
        # Đọc file CSV
        print(f"Đang đọc file {csv_file}...")
        df = pd.read_csv(csv_file)

        # Hiển thị thông tin ban đầu
        original_rows = len(df)
        print(f"Đã đọc {original_rows} dòng dữ liệu")

        # Chuyển đổi cột 'date' sang kiểu datetime để sắp xếp chính xác
        print("Đang chuyển đổi cột date...")
        df['date'] = pd.to_datetime(df['date'])

        # Sắp xếp dữ liệu theo cột date
        print("Đang sắp xếp dữ liệu...")
        df_sorted = df.sort_values(by='date')

        # Chuyển cột date trở lại định dạng chuỗi YYYY-MM-DD
        df_sorted['date'] = df_sorted['date'].dt.strftime('%Y-%m-%d')

        # Lưu file đã sắp xếp
        sorted_csv_file = f"{output_dir}/weather_data_sorted.csv"
        print(f"Đang lưu file đã sắp xếp vào {sorted_csv_file}...")
        df_sorted.to_csv(sorted_csv_file, index=False)

        # Thông tin thêm
        total_time = time.time() - start_time
        print(f"Hoàn thành! File đã được sắp xếp và lưu tại: {sorted_csv_file}")
        print(f"Tổng số dòng: {len(df_sorted)}")
        print(f"Tổng thời gian: {total_time:.2f} giây")

    except Exception as e:
        print(f"Lỗi khi sắp xếp file CSV: {e}")

In [ ]:
sort_csv_by_date()

In [ ]:
# Replace 'weather_data.csv' with your actual filename
data_path = 'weather_data.csv'

print("Loading dataset...")
try:
    df = pd.read_csv(data_path)
except Exception as e:
    print(f"Error loading with headers: {e}")
    print("Trying to load without headers...")

    column_names = [
        'city', 'region', 'lat', 'lon', 'date', 'date_epoch', 'maxtemp_c',
        'mintemp_c', 'avgtemp_c', 'maxwind_kph', 'totalprecip_mm', 'avgvis_km',
        'avghumidity', 'daily_will_it_rain', 'daily_chance_of_rain',
        'condition_text', 'condition_icon', 'condition_code', 'uv_mean',
        'sunrise', 'sunset', 'moonrise', 'moonset', 'moon_illumination'
    ]

    df = pd.read_csv(data_path, header=None, names=column_names)

# Convert date format if needed
df['date'] = pd.to_datetime(df['date'], errors='coerce')

# Step 2: Extract unique locations (lat, lon)
unique_locations = df[['city', 'region', 'lat', 'lon']].drop_duplicates().reset_index(drop=True)
print(f"Found {len(unique_locations)} unique locations")

# Create a cache directory for API responses
cache_dir = "cloud_cover_cache"
os.makedirs(cache_dir, exist_ok=True)

# Fixed date ranges - split into two parts:
# 1. Archive API for the early period
archive_start_date = "2021-01-01"
archive_end_date = "2021-03-23"

# 2. Regular historical forecast API for the later period
forecast_start_date = "2021-03-24"
forecast_end_date = "2025-06-03"

def fetch_cloud_cover(location_row):
    """
    Fetch cloud cover data for a specific location using both APIs
    Returns: Dictionary with location key and combined cloud cover data
    """
    idx, row = location_row
    city = row['city']
    region = row['region']
    lat = row['lat']
    lon = row['lon']

    # Create a unique key for this location
    location_key = f"{lat}_{lon}"

    # Create cache filenames for both APIs
    archive_cache_file = os.path.join(cache_dir, f"{location_key.replace('.', '_')}_archive.json")
    forecast_cache_file = os.path.join(cache_dir, f"{location_key.replace('.', '_')}_forecast.json")

    # Combined dictionary for results
    cloud_cover_dict = {}

    # Part 1: Get data from archive API for early period
    if os.path.exists(archive_cache_file):
        try:
            with open(archive_cache_file, 'r') as f:
                archive_data = json.load(f)
            cloud_cover_dict.update(archive_data)
        except Exception as e:
            print(f"Error reading archive cache for {city}: {e}")

    if archive_cache_file not in os.listdir(cache_dir):
        # Construct Archive API URL
        archive_url = f"https://archive-api.open-meteo.com/v1/archive?latitude={lat}&longitude={lon}&start_date={archive_start_date}&end_date={archive_end_date}&daily=cloud_cover_mean&timezone=Asia%2FBangkok"

        try:
            response = requests.get(archive_url)
            if response.status_code == 200:
                data = response.json()

                # Create a dictionary mapping dates to cloud cover values
                archive_result = {}
                for i, date_str in enumerate(data['daily']['time']):
                    archive_result[date_str] = data['daily']['cloud_cover_mean'][i]

                # Cache the result
                with open(archive_cache_file, 'w') as f:
                    json.dump(archive_result, f)

                # Add to combined results
                cloud_cover_dict.update(archive_result)
            else:
                print(f"Archive API request failed for {city}: Status code {response.status_code}")
        except Exception as e:
            print(f"Error fetching archive data for {city}: {e}")

    # Part 2: Get data from historical forecast API for later period
    if os.path.exists(forecast_cache_file):
        try:
            with open(forecast_cache_file, 'r') as f:
                forecast_data = json.load(f)
            cloud_cover_dict.update(forecast_data)
        except Exception as e:
            print(f"Error reading forecast cache for {city}: {e}")

    if forecast_cache_file not in os.listdir(cache_dir):
        # Construct Historical Forecast API URL
        forecast_url = f"https://historical-forecast-api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}&start_date={forecast_start_date}&end_date={forecast_end_date}&daily=cloud_cover_mean&timezone=Asia%2FBangkok"

        try:
            response = requests.get(forecast_url)
            if response.status_code == 200:
                data = response.json()

                # Create a dictionary mapping dates to cloud cover values
                forecast_result = {}
                for i, date_str in enumerate(data['daily']['time']):
                    forecast_result[date_str] = data['daily']['cloud_cover_mean'][i]

                # Cache the result
                with open(forecast_cache_file, 'w') as f:
                    json.dump(forecast_result, f)

                # Add to combined results
                cloud_cover_dict.update(forecast_result)
            else:
                print(f"Forecast API request failed for {city}: Status code {response.status_code}")
        except Exception as e:
            print(f"Error fetching forecast data for {city}: {e}")

    # Return the combined data
    return location_key, cloud_cover_dict

# Step 3: Fetch cloud cover data for all locations (with progress bar)
print("Fetching cloud cover data for all locations...")
all_cloud_data = {}

# Use ThreadPoolExecutor for parallel requests (adjust max_workers as needed)
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    # Submit all tasks and track with tqdm for progress
    futures = [executor.submit(fetch_cloud_cover, (idx, row)) for idx, row in unique_locations.iterrows()]

    for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures), desc="Fetching data"):
        try:
            location_key, cloud_data = future.result()
            all_cloud_data[location_key] = cloud_data
        except Exception as e:
            print(f"Error processing result: {e}")

# Step 4: Create a mapping dictionary for faster lookups
print("Creating location mapping...")
location_mapping = {}
for _, row in unique_locations.iterrows():
    location_key = f"{row['lat']}_{row['lon']}"
    location_mapping[(row['city'], row['region'])] = location_key

# Step 5: Add cloud cover data to the original dataframe
print("Adding cloud cover data to the dataset...")
df['cloud_cover_mean'] = None

# Vectorized operations for better performance
df['location_key'] = df.apply(lambda row: f"{row['lat']}_{row['lon']}", axis=1)
df['date_str'] = df['date'].dt.strftime('%Y-%m-%d')

# Apply the cloud cover values using a function
def get_cloud_cover(row):
    location_key = row['location_key']
    date_str = row['date_str']

    if location_key in all_cloud_data and date_str in all_cloud_data[location_key]:
        return all_cloud_data[location_key][date_str]
    return None

df['cloud_cover_mean'] = df.apply(get_cloud_cover, axis=1)

# Clean up temporary columns
df.drop(['location_key', 'date_str'], axis=1, inplace=True)

# Step 6: Save the enriched dataframe
output_path = 'weather_data_with_cloud_cover.csv'
print(f"Saving enriched data to {output_path}...")
df.to_csv(output_path, index=False)
print(f"Saved enriched data successfully!")

# Print sample of the data to verify
print("\nSample of the enriched data:")
print(df[['city', 'region', 'lat', 'lon', 'date', 'cloud_cover_mean']].head(10))

# Summary statistics for cloud cover
print("\nCloud cover summary statistics:")
print(df['cloud_cover_mean'].describe())

# Print coverage information
covered_records = df['cloud_cover_mean'].notna().sum()
print(f"\nCoverage: {covered_records}/{len(df)} records ({covered_records/len(df)*100:.2f}%)")

# Check coverage by time period
early_period = df[(df['date'] >= archive_start_date) & (df['date'] <= archive_end_date)]
later_period = df[(df['date'] > archive_end_date) & (df['date'] <= forecast_end_date)]

early_covered = early_period['cloud_cover_mean'].notna().sum()
later_covered = later_period['cloud_cover_mean'].notna().sum()